In [154]:
import scipy.stats as stats
import numpy as np
import sklearn as sk
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.ensemble import RandomForestRegressor
from matplotlib import pyplot as plt
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [155]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

# Load your dataset
df = pd.read_csv('./data/BPAT/BPAT_direct_emissions.csv')

# Ensure 'UTC time' is in datetime format
df['UTC time'] = pd.to_datetime(df['UTC time'])

# Filter out February 29 for the leap year (2020) to ensure alignment
df = df[~((df['UTC time'].dt.year == 2020) & 
          (df['UTC time'].dt.month == 2) & 
          (df['UTC time'].dt.day == 29))]

# Specify the training and testing years
train_year = 2020
test_year = 2021

# Filter for the training year and prepare training data
train_data = df[df['UTC time'].dt.year == train_year]
X_train = train_data[['nat_gas', 'carbon_intensity']].values[:-24]  # Feature for training
y_train = train_data['carbon_intensity'].values[24:].reshape(-1, 1)  # Target for training

# Filter for the testing year and prepare testing data
test_data = df[df['UTC time'].dt.year == test_year]
X_test = test_data[['nat_gas', 'carbon_intensity']].values[:-24]  # Feature for testing
y_test = test_data['carbon_intensity'].values[24:].reshape(-1, 1)  # Target for testing

# Print column names for verification
print("Training Features (X_train columns):", ['nat_gas', 'carbon_intensity'])
print("Training Target (y_train column): ['carbon_intensity']")
print("Testing Features (X_test columns):", ['nat_gas', 'carbon_intensity'])
print("Testing Target (y_test column): ['carbon_intensity']")
print("\n")

# Train the model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test[:-24])

# Evaluate the model
rmse = np.sqrt(mean_squared_error(y_test[24:], y_pred))
mape = mean_absolute_percentage_error(y_test[24:], y_pred)

# Print the results
print("Results for Energy Source: nat_gas")
print(f"  RMSE: {rmse}")
print(f"  MAPE: {mape}\n")

Training Features (X_train columns): ['nat_gas', 'carbon_intensity']
Training Target (y_train column): ['carbon_intensity']
Testing Features (X_test columns): ['nat_gas', 'carbon_intensity']
Testing Target (y_test column): ['carbon_intensity']


Results for Energy Source: nat_gas
  RMSE: 8.407486784204801
  MAPE: 0.17569031145874336



In [156]:
# Load your dataset
df = pd.read_csv('./data/BPAT/BPAT_direct_emissions.csv')

# Ensure 'UTC time' is in datetime format
df['UTC time'] = pd.to_datetime(df['UTC time'])

# Filter out February 29 for the leap year (2020) to ensure alignment
df = df[~((df['UTC time'].dt.year == 2020) & 
          (df['UTC time'].dt.month == 2) & 
          (df['UTC time'].dt.day == 29))]

# Specify the training and testing years
train_year = 2020
test_year = 2021

# Filter for the training year and prepare training data
train_data = df[df['UTC time'].dt.year == train_year]
X_train = train_data[['nat_gas', 'carbon_intensity', 'nuclear']].values[:-24]  # Feature for training
y_train = train_data['carbon_intensity'].values[24:].reshape(-1, 1)  # Target for training

# Filter for the testing year and prepare testing data
test_data = df[df['UTC time'].dt.year == test_year]
X_test = test_data[['nat_gas', 'carbon_intensity', 'nuclear']].values[:-24]  # Feature for testing
y_test = test_data['carbon_intensity'].values[24:].reshape(-1, 1)  # Target for testing

# Print column names for verification
print("Training Features (X_train columns):", ['nat_gas', 'carbon_intensity', 'nuclear'])
print("Training Target (y_train column): ['carbon_intensity']")
print("Testing Features (X_test columns):", ['nat_gas', 'carbon_intensity', 'nuclear'])
print("Testing Target (y_test column): ['carbon_intensity']")
print("\n")

# Train the model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test[:-24])

# Evaluate the model
rmse = np.sqrt(mean_squared_error(y_test[24:], y_pred))
mape = mean_absolute_percentage_error(y_test[24:], y_pred)

# Print the results
print("Results for Energy Source: nat_gas")
print(f"  RMSE: {rmse}")
print(f"  MAPE: {mape}\n")

Training Features (X_train columns): ['nat_gas', 'carbon_intensity', 'nuclear']
Training Target (y_train column): ['carbon_intensity']
Testing Features (X_test columns): ['nat_gas', 'carbon_intensity', 'nuclear']
Testing Target (y_test column): ['carbon_intensity']


Results for Energy Source: nat_gas
  RMSE: 8.395113409299668
  MAPE: 0.1761367677721789



# Doing multivariate linear regression with only 2 features for all the energy sources

In [157]:
def prepare_data(df, train_year, test_year, feature_column):
    # Ensure datetime is in datetime format
    df['UTC time'] = pd.to_datetime(df['UTC time'])
    
    # Filter out February 29 for the leap year (2020) to ensure alignment
    df = df[~((df['UTC time'].dt.year == 2020) & 
              (df['UTC time'].dt.month == 2) & 
              (df['UTC time'].dt.day == 29))]
    
    # Filter for the training year
    train_data = df[df['UTC time'].dt.year == train_year]
    X_train = train_data[[feature_column, 'carbon_intensity']].values[:-24]  # Exclude last 24 rows
    y_train = train_data['carbon_intensity'].values[24:].reshape(-1, 1)  # Exclude first 24 rows to align

    # Filter for the testing year
    test_data = df[df['UTC time'].dt.year == test_year]
    X_test = test_data[[feature_column, 'carbon_intensity']].values[:-24]  # Use all rows for testing features
    y_test = test_data['carbon_intensity'].values[24:].reshape(-1, 1)  # Use all rows for testing target
    
    # Print column names
    print("Training Features (X_train columns):", [feature_column, 'carbon_intensity'])
    print("Training Target (y_train column): ['carbon_intensity']")
    print("Testing Features (X_test columns):", [feature_column, 'carbon_intensity'])
    print("Testing Target (y_test column): ['carbon_intensity']")
    print("\n")
    
    return X_train, y_train, X_test, y_test

def evaluate_energy_sources(df, train_year, test_year, energy_sources):
    results = []

    for feature_column in energy_sources:
        # Prepare data for the current energy source
        X_train, y_train, X_test, y_test = prepare_data(df, train_year, test_year, feature_column)
        
        print("X_train data", X_train[:5])
        
        # Train the model
        model = LinearRegression()
        model.fit(X_train, y_train)
        
        # Predict on the test set, aligning with 24-hour shift
        y_pred = model.predict(X_test[:-24])
        
        # Evaluate the model using corresponding `y_test` values from the 24th row onward
        rmse = np.sqrt(mean_squared_error(y_test[24:], y_pred))
        mape = mean_absolute_percentage_error(y_test[24:], y_pred)
        
        # Store the results
        results.append({
            "Energy Source": feature_column,
            "RMSE": rmse,
            "MAPE": mape
        })
        
        # Print the results for each energy source
        print(f"Energy Source: {feature_column}")
        print(f"  RMSE: {rmse}")
        print(f"  MAPE: {mape}\n")
    
    # Convert results to DataFrame for easy comparison and display all results
    results_df = pd.DataFrame(results)
    print("All Results for Each Energy Source as a feature with carbon_intensty:")
    print(results_df)
    
    return results_df

# Load your dataset
df = pd.read_csv('./data/BPAT/BPAT_direct_emissions.csv')

# Specify the training and testing years
train_year = 2020
test_year = 2021

# List of energy sources to evaluate
energy_sources = ['nat_gas', 'nuclear', 'hydro', 'solar', 'wind', 'other']

# Evaluate each energy source and display results
results_df = evaluate_energy_sources(df, train_year, test_year, energy_sources)
min_mape = results_df['MAPE'].min()
max_mape = results_df['MAPE'].max()

print(f"\nMinimum MAPE: {min_mape}")
print(f"Maximum MAPE: {max_mape}")


Training Features (X_train columns): ['nat_gas', 'carbon_intensity']
Training Target (y_train column): ['carbon_intensity']
Testing Features (X_test columns): ['nat_gas', 'carbon_intensity']
Testing Target (y_test column): ['carbon_intensity']


X_train data [[888.    39.34]
 [890.    38.05]
 [886.    37.65]
 [887.    37.36]
 [885.    39.36]]
Energy Source: nat_gas
  RMSE: 8.407486784204801
  MAPE: 0.17569031145874336

Training Features (X_train columns): ['nuclear', 'carbon_intensity']
Training Target (y_train column): ['carbon_intensity']
Testing Features (X_test columns): ['nuclear', 'carbon_intensity']
Testing Target (y_test column): ['carbon_intensity']


X_train data [[1148.     39.34]
 [1151.     38.05]
 [1152.     37.65]
 [1152.     37.36]
 [1155.     39.36]]
Energy Source: nuclear
  RMSE: 8.398420927663631
  MAPE: 0.17615126330904238

Training Features (X_train columns): ['hydro', 'carbon_intensity']
Training Target (y_train column): ['carbon_intensity']
Testing Features (X_te

# Doing multivariate linear regression with only 3 features for all the energy sources, i.e 2 energy sources and carbon intensity

In [158]:
from itertools import combinations
import math

def prepare_data(df, train_year, test_year, feature_columns):
    # Ensure datetime is in datetime format
    df['UTC time'] = pd.to_datetime(df['UTC time'])
    
    # Filter out February 29 for the leap year (2020) to ensure alignment
    df = df[~((df['UTC time'].dt.year == 2020) & 
              (df['UTC time'].dt.month == 2) & 
              (df['UTC time'].dt.day == 29))]
    
    # Filter for the training year
    train_data = df[df['UTC time'].dt.year == train_year]
    X_train = train_data[feature_columns + ['carbon_intensity']].values[:-24]  # Exclude last 24 rows
    y_train = train_data['carbon_intensity'].values[24:].reshape(-1, 1)  # Exclude first 24 rows to align

    # Filter for the testing year
    test_data = df[df['UTC time'].dt.year == test_year]
    X_test = test_data[feature_columns + ['carbon_intensity']].values[:-24]  # Use all rows for testing features
    y_test = test_data['carbon_intensity'].values[24:].reshape(-1, 1)  # Use all rows for testing target
    
    # Print column names
    print("Training Features (X_train columns):", feature_columns + ['carbon_intensity'])
    print("Training Target (y_train column): ['carbon_intensity']")
    print("Testing Features (X_test columns):", feature_columns + ['carbon_intensity'])
    print("Testing Target (y_test column): ['carbon_intensity']")
    print("\n")
    
    return X_train, y_train, X_test, y_test

def evaluate_energy_sources(df, train_year, test_year, energy_sources):
    results = []

    # Generate all combinations of two energy sources
    for feature_columns in combinations(energy_sources, 2):
        feature_columns = list(feature_columns)  # Convert tuple to list for processing
        
        # Prepare data for the current pair of energy sources plus 'carbon_intensity'
        X_train, y_train, X_test, y_test = prepare_data(df, train_year, test_year, feature_columns)
        
        print("X_train data", X_train[:5])
        
        # Train the model
        model = LinearRegression()
        model.fit(X_train, y_train)
        
        # Predict on the test set, aligning with 24-hour shift
        y_pred = model.predict(X_test[:-24])
        
        # Evaluate the model using corresponding `y_test` values from the 24th row onward
        rmse = np.sqrt(mean_squared_error(y_test[24:], y_pred))
        mape = mean_absolute_percentage_error(y_test[24:], y_pred)
        
        # Store the results
        results.append({
            "Energy Sources": f"{feature_columns[0]} + {feature_columns[1]} + carbon_intensity",
            "RMSE": rmse,
            "MAPE": mape
        })
        
        # Print the results for each energy source combination
        print(f"Energy Sources: {feature_columns[0]}, {feature_columns[1]} + carbon_intensity")
        print(f"  RMSE: {rmse}")
        print(f"  MAPE: {mape}\n")
    
    # Convert results to DataFrame for easy comparison and display all results
    results_df = pd.DataFrame(results)
    print("All Results for Each Combination of 2 Energy Sources with carbon_intensity:")
    print(results_df)
    
    return results_df

# Load your dataset
df = pd.read_csv('./data/BPAT/BPAT_direct_emissions.csv')

# Specify the training and testing years
train_year = 2020
test_year = 2021

# List of energy sources to evaluate
energy_sources = ['nat_gas', 'nuclear', 'hydro', 'solar', 'wind', 'other']

# Evaluate each combination of 2 energy sources with 'carbon_intensity' and display results
results_df = evaluate_energy_sources(df, train_year, test_year, energy_sources)
min_mape = results_df['MAPE'].min()
max_mape = results_df['MAPE'].max()

print(f"\nMinimum MAPE: {min_mape}")
print(f"Maximum MAPE: {max_mape}")
print("possible combinations with 6 choose 2 is:", math.comb(6, 2))

Training Features (X_train columns): ['nat_gas', 'nuclear', 'carbon_intensity']
Training Target (y_train column): ['carbon_intensity']
Testing Features (X_test columns): ['nat_gas', 'nuclear', 'carbon_intensity']
Testing Target (y_test column): ['carbon_intensity']


X_train data [[ 888.   1148.     39.34]
 [ 890.   1151.     38.05]
 [ 886.   1152.     37.65]
 [ 887.   1152.     37.36]
 [ 885.   1155.     39.36]]
Energy Sources: nat_gas, nuclear + carbon_intensity
  RMSE: 8.395113409299666
  MAPE: 0.1761367677721786

Training Features (X_train columns): ['nat_gas', 'hydro', 'carbon_intensity']
Training Target (y_train column): ['carbon_intensity']
Testing Features (X_test columns): ['nat_gas', 'hydro', 'carbon_intensity']
Testing Target (y_test column): ['carbon_intensity']


X_train data [[ 888.   7760.     39.34]
 [ 890.   8057.     38.05]
 [ 886.   8064.     37.65]
 [ 887.   8418.     37.36]
 [ 885.   7858.     39.36]]
Energy Sources: nat_gas, hydro + carbon_intensity
  RMSE: 8.1082

# Doing multivariate linear regression with only 4 features for all the energy sources, i.e 3 energy sources and carbon intensity

In [159]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from itertools import combinations

def prepare_data(df, train_year, test_year, feature_columns):
    # Ensure datetime is in datetime format
    df['UTC time'] = pd.to_datetime(df['UTC time'])
    
    # Filter out February 29 for the leap year (2020) to ensure alignment
    df = df[~((df['UTC time'].dt.year == 2020) & 
              (df['UTC time'].dt.month == 2) & 
              (df['UTC time'].dt.day == 29))]
    
    # Filter for the training year
    train_data = df[df['UTC time'].dt.year == train_year]
    X_train = train_data[feature_columns + ['carbon_intensity']].values[:-24]  # Exclude last 24 rows
    y_train = train_data['carbon_intensity'].values[24:].reshape(-1, 1)  # Exclude first 24 rows to align

    # Filter for the testing year
    test_data = df[df['UTC time'].dt.year == test_year]
    X_test = test_data[feature_columns + ['carbon_intensity']].values[:-24]  # Use all rows for testing features
    y_test = test_data['carbon_intensity'].values[24:].reshape(-1, 1)  # Use all rows for testing target
    
    # Print column names
    print("Training Features (X_train columns):", feature_columns + ['carbon_intensity'])
    print("Training Target (y_train column): ['carbon_intensity']")
    print("Testing Features (X_test columns):", feature_columns + ['carbon_intensity'])
    print("Testing Target (y_test column): ['carbon_intensity']")
    print("\n")
    
    return X_train, y_train, X_test, y_test

def evaluate_energy_sources(df, train_year, test_year, energy_sources):
    results = []

    # Generate all combinations of three energy sources
    for feature_columns in combinations(energy_sources, 3):
        feature_columns = list(feature_columns)  # Convert tuple to list for processing
        
        # Prepare data for the current triplet of energy sources plus 'carbon_intensity'
        X_train, y_train, X_test, y_test = prepare_data(df, train_year, test_year, feature_columns)
        
        print("X_train data", X_train[:5])
        
        # Train the model
        model = LinearRegression()
        model.fit(X_train, y_train)
        
        # Predict on the test set, aligning with 24-hour shift
        y_pred = model.predict(X_test[:-24])
        
        # Evaluate the model using corresponding `y_test` values from the 24th row onward
        rmse = np.sqrt(mean_squared_error(y_test[24:], y_pred))
        mape = mean_absolute_percentage_error(y_test[24:], y_pred)
        
        # Store the results
        results.append({
            "Energy Sources": f"{feature_columns[0]}, {feature_columns[1]}, {feature_columns[2]} + carbon_intensity",
            "RMSE": rmse,
            "MAPE": mape
        })
        
        # Print the results for each combination of energy sources
        print(f"Energy Sources: {feature_columns[0]}, {feature_columns[1]}, {feature_columns[2]} + carbon_intensity")
        print(f"  RMSE: {rmse}")
        print(f"  MAPE: {mape}\n")
    
    # Convert results to DataFrame for easy comparison and display all results
    results_df = pd.DataFrame(results)
    print("All Results for Each Combination of 3 Energy Sources with carbon_intensity:")
    print(results_df)
    
    return results_df

# Load your dataset
df = pd.read_csv('./data/BPAT/BPAT_direct_emissions.csv')

# Specify the training and testing years
train_year = 2020
test_year = 2021

# List of energy sources to evaluate
energy_sources = ['nat_gas', 'nuclear', 'hydro', 'solar', 'wind', 'other']

# Evaluate each combination of 3 energy sources with 'carbon_intensity' and display results
results_df = evaluate_energy_sources(df, train_year, test_year, energy_sources)
min_mape = results_df['MAPE'].min()
max_mape = results_df['MAPE'].max()

print(f"\nMinimum MAPE: {min_mape}")
print(f"Maximum MAPE: {max_mape}")
print("possible combinations with 6 choose 3 is: ", math.comb(6, 3))

Training Features (X_train columns): ['nat_gas', 'nuclear', 'hydro', 'carbon_intensity']
Training Target (y_train column): ['carbon_intensity']
Testing Features (X_test columns): ['nat_gas', 'nuclear', 'hydro', 'carbon_intensity']
Testing Target (y_test column): ['carbon_intensity']


X_train data [[ 888.   1148.   7760.     39.34]
 [ 890.   1151.   8057.     38.05]
 [ 886.   1152.   8064.     37.65]
 [ 887.   1152.   8418.     37.36]
 [ 885.   1155.   7858.     39.36]]
Energy Sources: nat_gas, nuclear, hydro + carbon_intensity
  RMSE: 8.124331194982856
  MAPE: 0.18189627450777995

Training Features (X_train columns): ['nat_gas', 'nuclear', 'solar', 'carbon_intensity']
Training Target (y_train column): ['carbon_intensity']
Testing Features (X_test columns): ['nat_gas', 'nuclear', 'solar', 'carbon_intensity']
Testing Target (y_test column): ['carbon_intensity']


X_train data [[ 888.   1148.      0.     39.34]
 [ 890.   1151.      0.     38.05]
 [ 886.   1152.      0.     37.65]
 [ 887.

# Doing multivariate linear regression with only 5 features for all the energy sources, i.e 4 energy sources and carbon intensity

In [160]:
def prepare_data(df, train_year, test_year, feature_columns):
    # Ensure datetime is in datetime format
    df['UTC time'] = pd.to_datetime(df['UTC time'])
    
    # Filter out February 29 for the leap year (2020) to ensure alignment
    df = df[~((df['UTC time'].dt.year == 2020) & 
              (df['UTC time'].dt.month == 2) & 
              (df['UTC time'].dt.day == 29))]
    
    # Filter for the training year
    train_data = df[df['UTC time'].dt.year == train_year]
    X_train = train_data[feature_columns + ['carbon_intensity']].values[:-24]  # Exclude last 24 rows
    y_train = train_data['carbon_intensity'].values[24:].reshape(-1, 1)  # Exclude first 24 rows to align

    # Filter for the testing year
    test_data = df[df['UTC time'].dt.year == test_year]
    X_test = test_data[feature_columns + ['carbon_intensity']].values[:-24]  # Use all rows for testing features
    y_test = test_data['carbon_intensity'].values[24:].reshape(-1, 1)  # Use all rows for testing target
    
    return X_train, y_train, X_test, y_test

def evaluate_energy_sources(df, train_year, test_year, energy_sources):
    results = []

    # Generate all combinations of four energy sources
    for feature_columns in combinations(energy_sources, 4):
        feature_columns = list(feature_columns)  # Convert tuple to list for processing
        
        # Prepare data for the current set of four energy sources plus 'carbon_intensity'
        X_train, y_train, X_test, y_test = prepare_data(df, train_year, test_year, feature_columns)
        
        # Train the model
        model = LinearRegression()
        model.fit(X_train, y_train)
        
        # Predict on the test set, aligning with 24-hour shift
        y_pred = model.predict(X_test[:-24])
        
        # Evaluate the model using corresponding `y_test` values from the 24th row onward
        rmse = np.sqrt(mean_squared_error(y_test[24:], y_pred))
        mape = mean_absolute_percentage_error(y_test[24:], y_pred)
        
        # Store the results
        results.append({
            "Energy Sources": f"{feature_columns[0]}, {feature_columns[1]}, {feature_columns[2]}, {feature_columns[3]} + carbon_intensity",
            "RMSE": rmse,
            "MAPE": mape
        })
        
        # Print the results for each combination of energy sources
        print(f"Energy Sources: {feature_columns[0]}, {feature_columns[1]}, {feature_columns[2]}, {feature_columns[3]} + carbon_intensity")
        print(f"  RMSE: {rmse}")
        print(f"  MAPE: {mape}\n")
    
    # Convert results to DataFrame for easy comparison and display all results
    results_df = pd.DataFrame(results)
    print("All Results for Each Combination of 4 Energy Sources with carbon_intensity:")
    print(results_df)
    
    # Find and print the minimum and maximum MAPE values
    min_mape = results_df['MAPE'].min()
    max_mape = results_df['MAPE'].max()
    
    min_mape_row = results_df.loc[results_df['MAPE'].idxmin()]
    max_mape_row = results_df.loc[results_df['MAPE'].idxmax()]
    
    print(f"\nCombination with Minimum MAPE:\n{min_mape_row}")
    print(f"\nCombination with Maximum MAPE:\n{max_mape_row}")

    return results_df

# Load your dataset
df = pd.read_csv('./data/BPAT/BPAT_direct_emissions.csv')

# Specify the training and testing years
train_year = 2020
test_year = 2021

# List of energy sources to evaluate
energy_sources = ['nat_gas', 'nuclear', 'hydro', 'solar', 'wind', 'other']

# Evaluate each combination of 4 energy sources with 'carbon_intensity' and display results
results_df = evaluate_energy_sources(df, train_year, test_year, energy_sources)


Energy Sources: nat_gas, nuclear, hydro, solar + carbon_intensity
  RMSE: 8.128632007923324
  MAPE: 0.18271171929614397

Energy Sources: nat_gas, nuclear, hydro, wind + carbon_intensity
  RMSE: 8.043158183036446
  MAPE: 0.1768537222555926

Energy Sources: nat_gas, nuclear, hydro, other + carbon_intensity
  RMSE: 8.124216851913141
  MAPE: 0.18277746486790933

Energy Sources: nat_gas, nuclear, solar, wind + carbon_intensity
  RMSE: 8.117674076749047
  MAPE: 0.17273311865295343

Energy Sources: nat_gas, nuclear, solar, other + carbon_intensity
  RMSE: 8.383741949704401
  MAPE: 0.17589793675345392

Energy Sources: nat_gas, nuclear, wind, other + carbon_intensity
  RMSE: 8.11801731555379
  MAPE: 0.17237006955174788

Energy Sources: nat_gas, hydro, solar, wind + carbon_intensity
  RMSE: 8.051713649962009
  MAPE: 0.17819937875166844

Energy Sources: nat_gas, hydro, solar, other + carbon_intensity
  RMSE: 8.105738125053122
  MAPE: 0.18152617345941774

Energy Sources: nat_gas, hydro, wind, othe

# Doing multivariate linear regression with only 6 features for all the energy sources, i.e 5 energy sources and carbon intensity

In [161]:
def prepare_data(df, train_year, test_year, feature_columns):
    # Ensure datetime is in datetime format
    df['UTC time'] = pd.to_datetime(df['UTC time'])
    
    # Filter out February 29 for the leap year (2020) to ensure alignment
    df = df[~((df['UTC time'].dt.year == 2020) & 
              (df['UTC time'].dt.month == 2) & 
              (df['UTC time'].dt.day == 29))]
    
    # Filter for the training year
    train_data = df[df['UTC time'].dt.year == train_year]
    X_train = train_data[feature_columns + ['carbon_intensity']].values[:-24]  # Exclude last 24 rows
    y_train = train_data['carbon_intensity'].values[24:].reshape(-1, 1)  # Exclude first 24 rows to align

    # Filter for the testing year
    test_data = df[df['UTC time'].dt.year == test_year]
    X_test = test_data[feature_columns + ['carbon_intensity']].values[:-24]  # Use all rows for testing features
    y_test = test_data['carbon_intensity'].values[24:].reshape(-1, 1)  # Use all rows for testing target
    
    return X_train, y_train, X_test, y_test

def evaluate_energy_sources(df, train_year, test_year, energy_sources):
    results = []

    # Generate all combinations of five energy sources
    for feature_columns in combinations(energy_sources, 5):
        feature_columns = list(feature_columns)  # Convert tuple to list for processing
        
        # Prepare data for the current set of five energy sources plus 'carbon_intensity'
        X_train, y_train, X_test, y_test = prepare_data(df, train_year, test_year, feature_columns)
        
        # Train the model
        model = LinearRegression()
        model.fit(X_train, y_train)
        
        # Predict on the test set, aligning with 24-hour shift
        y_pred = model.predict(X_test[:-24])
        
        # Evaluate the model using corresponding `y_test` values from the 24th row onward
        rmse = np.sqrt(mean_squared_error(y_test[24:], y_pred))
        mape = mean_absolute_percentage_error(y_test[24:], y_pred)
        
        # Store the results
        results.append({
            "Energy Sources": f"{feature_columns[0]}, {feature_columns[1]}, {feature_columns[2]}, {feature_columns[3]}, {feature_columns[4]} + carbon_intensity",
            "RMSE": rmse,
            "MAPE": mape
        })
        
        # Print the results for each combination of energy sources
        print(f"Energy Sources: {feature_columns[0]}, {feature_columns[1]}, {feature_columns[2]}, {feature_columns[3]}, {feature_columns[4]} + carbon_intensity")
        print(f"  RMSE: {rmse}")
        print(f"  MAPE: {mape}\n")
    
    # Convert results to DataFrame for easy comparison and display all results
    results_df = pd.DataFrame(results)
    print("All Results for Each Combination of 5 Energy Sources with carbon_intensity:")
    print(results_df)
    
    # Find and print the minimum and maximum MAPE values
    min_mape = results_df['MAPE'].min()
    max_mape = results_df['MAPE'].max()
    
    min_mape_row = results_df.loc[results_df['MAPE'].idxmin()]
    max_mape_row = results_df.loc[results_df['MAPE'].idxmax()]
    
    print(f"\nCombination with Minimum MAPE:\n{min_mape_row}")
    print(f"\nCombination with Maximum MAPE:\n{max_mape_row}")

    return results_df

# Load your dataset
df = pd.read_csv('./data/BPAT/BPAT_direct_emissions.csv')

# Specify the training and testing years
train_year = 2020
test_year = 2021

# List of energy sources to evaluate
energy_sources = ['nat_gas', 'nuclear', 'hydro', 'solar', 'wind', 'other']

# Evaluate each combination of 5 energy sources with 'carbon_intensity' and display results
results_df = evaluate_energy_sources(df, train_year, test_year, energy_sources)


Energy Sources: nat_gas, nuclear, hydro, solar, wind + carbon_intensity
  RMSE: 8.044509062037232
  MAPE: 0.17753493375942841

Energy Sources: nat_gas, nuclear, hydro, solar, other + carbon_intensity
  RMSE: 8.131603050186989
  MAPE: 0.18396912149837566

Energy Sources: nat_gas, nuclear, hydro, wind, other + carbon_intensity
  RMSE: 8.040019664598866
  MAPE: 0.17736470686157285

Energy Sources: nat_gas, nuclear, solar, wind, other + carbon_intensity
  RMSE: 8.110421897154612
  MAPE: 0.17261658921059836

Energy Sources: nat_gas, hydro, solar, wind, other + carbon_intensity
  RMSE: 8.044027376739272
  MAPE: 0.1784481060134097

Energy Sources: nuclear, hydro, solar, wind, other + carbon_intensity
  RMSE: 8.077880371199342
  MAPE: 0.17600465698130932

All Results for Each Combination of 5 Energy Sources with carbon_intensity:
                                      Energy Sources      RMSE      MAPE
0  nat_gas, nuclear, hydro, solar, wind + carbon_...  8.044509  0.177535
1  nat_gas, nuclear,

# Doing multivariate linear regression with only 7 features for all the energy sources, i.e all 6 energy sources and carbon intensity

In [162]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

def prepare_data(df, train_year, test_year, feature_columns):
    # Ensure datetime is in datetime format
    df['UTC time'] = pd.to_datetime(df['UTC time'])
    
    # Filter out February 29 for the leap year (2020) to ensure alignment
    df = df[~((df['UTC time'].dt.year == 2020) & 
              (df['UTC time'].dt.month == 2) & 
              (df['UTC time'].dt.day == 29))]
    
    # Filter for the training year
    train_data = df[df['UTC time'].dt.year == train_year]
    X_train = train_data[feature_columns + ['carbon_intensity']].values[:-24]  # Exclude last 24 rows
    y_train = train_data['carbon_intensity'].values[24:].reshape(-1, 1)  # Exclude first 24 rows to align

    # Filter for the testing year
    test_data = df[df['UTC time'].dt.year == test_year]
    X_test = test_data[feature_columns + ['carbon_intensity']].values[:-24]  # Use all rows for testing features
    y_test = test_data['carbon_intensity'].values[24:].reshape(-1, 1)  # Use all rows for testing target
    
    return X_train, y_train, X_test, y_test

def evaluate_all_energy_sources(df, train_year, test_year):
    # Define the list of all energy sources
    energy_sources = ['nat_gas', 'nuclear', 'hydro', 'solar', 'wind', 'other']
    
    # Use all six energy sources + 'carbon_intensity' as features
    X_train, y_train, X_test, y_test = prepare_data(df, train_year, test_year, energy_sources)
    
    # Train the model
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # Predict on the test set, aligning with 24-hour shift
    y_pred = model.predict(X_test[:-24])
    
    # Evaluate the model using corresponding `y_test` values from the 24th row onward
    rmse = np.sqrt(mean_squared_error(y_test[24:], y_pred))
    mape = mean_absolute_percentage_error(y_test[24:], y_pred)
    
    # Print the results
    print("All Six Energy Sources + carbon_intensity as Features:")
    print(f"  RMSE: {rmse}")
    print(f"  MAPE: {mape}\n")

# Load your dataset
df = pd.read_csv('./data/BPAT/BPAT_direct_emissions.csv')

# Specify the training and testing years
train_year = 2020
test_year = 2021

# Evaluate the model using all energy sources and carbon_intensity as features
evaluate_all_energy_sources(df, train_year, test_year)


All Six Energy Sources + carbon_intensity as Features:
  RMSE: 8.043010843736456
  MAPE: 0.17834810618588043

